# NYC Schools Analysis using SQL and Python

This notebook connects to a PostgreSQL database hosted on Neon that contains
NYC high school data. Using SQL and Python (pandas), we will:

- Connect to the database using SQLAlchemy
- Explore the available tables and their schemas
- Answer key analytical questions:
  1. How many schools are there in each borough?
  2. What is the average percentage of English Language Learners (ELL) per borough?
  3. Which are the top 3 schools in each borough with the highest percentage of
     special education students (sped_percent)?
- Summarize key insights in Markdown cells.


In [2]:
# 1. Imports

import pandas as pd
from sqlalchemy import create_engine

## 2. Database Connection

We connect to the PostgreSQL database using SQLAlchemy.

In [4]:
# 2. Database Connection Setup (Neon PostgreSQL)

DATABASE_URL = (
    "postgresql+psycopg2://neondb_owner:a9Am7Yy5r9_T7h4OF2GN"
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech:5432/neondb"
    "?sslmode=require"
)

# Create the SQLAlchemy engine
engine = create_engine(DATABASE_URL)

# Quick test query
test_query = "SELECT 1 AS test_column;"
test_df = pd.read_sql(test_query, engine)
test_df

,test_column
0,1


## 3. Helper Function for Running SQL Queries

To keep the notebook clean and reusable, we define a small helper function
that sends a SQL query to the database and returns the result as a pandas DataFrame.

In [5]:
# 3. Helper Function

def run_query(sql: str) -> pd.DataFrame:
    """
    Execute a SQL query using the global SQLAlchemy engine
    and return the result as a pandas DataFrame.
    """
    return pd.read_sql(sql, engine)

## 4. Explore the Database Schema

Before writing analytical queries, we first explore:
- Which tables are available
- The columns in the key tables:
  - `nyc_schools.high_school_directory`
  - `nyc_schools.school_demographics`
  - `nyc_schools.school_safety_report`

In [6]:
# 4.1 List tables in the nyc_schools schema

tables_sql = """
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema = 'nyc_schools'
ORDER BY table_name;
"""

tables_df = run_query(tables_sql)
tables_df

,table_schema,table_name
0,nyc_schools,Essam_alasaad_sat_results
1,nyc_schools,abida_sultana_sat_scores
2,nyc_schools,anastasia_sat_results
3,nyc_schools,bianca_sat_results
4,nyc_schools,darel-kigha_sat_results
5,nyc_schools,dido_sat_results
6,nyc_schools,hakim-murphy_sat_results
7,nyc_schools,heike_reichert_sat_results
8,nyc_schools,high_school_directory
9,nyc_schools,isabella_leach_sat_results


In [7]:
# 4.2 Inspect columns of high_school_directory

hs_dir_cols_sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'nyc_schools'
  AND table_name = 'high_school_directory'
ORDER BY ordinal_position;
"""

hs_dir_cols_df = run_query(hs_dir_cols_sql)
hs_dir_cols_df

,column_name,data_type
0,dbn,text
1,school_name,text
2,borough,text
3,building_code,text
4,phone_number,text
...,...,...
100,Zip Codes,character varying
101,Community Districts,character varying
102,Borough Boundaries,character varying
103,City Council Districts,character varying


In [8]:
# 4.3 Inspect columns of school_demographics

demo_cols_sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'nyc_schools'
  AND table_name = 'school_demographics'
ORDER BY ordinal_position;
"""

demo_cols_df = run_query(demo_cols_sql)
demo_cols_df

,column_name,data_type
0,dbn,character varying
1,Name,character varying
2,schoolyear,integer
3,fl_percent,character varying
4,frl_percent,real
5,total_enrollment,integer
6,prek,character varying
7,k,character varying
8,grade1,character varying
9,grade2,character varying


In [20]:
# 4.4 Inspect columns of school_safety_report

safety_cols_sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'nyc_schools'
  AND table_name = 'school_safety_report'
ORDER BY ordinal_position;
"""

safety_cols_df = run_query(safety_cols_sql)
safety_cols_df

,column_name,data_type
0,school_year,text
1,building_code,text
2,dbn,text
3,location_name,text
4,location_code,text
5,address,text
6,borough,text
7,geographical_district_code,double precision
8,register,text
9,building_name,text


## 5. Analytical Questions

In this section, we address the three main analytical questions using SQL queries
and analyze the results using pandas.

### 5.1 Question 1 – School Distribution: How many schools are there in each borough?

We want to understand how schools are distributed geographically across NYC boroughs.
To do this, we:

- Use the `nyc_schools.high_school_directory` table
- Group by `borough`
- Count the number of schools per borough

In [9]:
# 5.1 Number of unique schools by borough

q1_sql = """
SELECT
    borough,
    COUNT(DISTINCT dbn) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY school_count DESC;
"""

q1_df = run_query(q1_sql)
q1_df

,borough,school_count
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10


#### Interpretation (Q1)

- The table above shows, for each borough, the **number of unique schools**,
  based on distinct `dbn` values in the high school directory.
- By counting `DISTINCT dbn`, we avoid double-counting schools that may appear
  multiple times in the directory due to multiple programs or other details.
- This provides a more accurate picture of how high schools are distributed
  geographically across NYC and can be used as a starting point for thinking
  about resource allocation and borough-level coverage.

### 5.2 – Average ELL Percentage by Borough (Full Dataset)

In this section, we compute the **average percentage of English Language Learners (ELL)**
for each borough using the entire `school_demographics` dataset.

Our approach:

1. Use `nyc_schools.school_demographics` as the source of ELL data (`ell_percent`).
2. Join it with `nyc_schools.high_school_directory` on the common school identifier `dbn`
   in order to obtain the corresponding `borough`.
3. Group the joined data by `borough`.
4. Compute the average `ell_percent` for each borough across all available records.

In [10]:
# 5.2 Average ELL percentage by borough using the full demographics dataset

q2_sql = """
SELECT
    d.borough,
    AVG(s.ell_percent) AS avg_ell_percent
FROM nyc_schools.school_demographics AS s
JOIN nyc_schools.high_school_directory AS d
    ON s.dbn = d.dbn
GROUP BY d.borough
ORDER BY d.borough;
"""

q2_df = run_query(q2_sql)
q2_df

,borough,avg_ell_percent
0,Manhattan,7.5725


#### Interpretation (Q2 – Full Dataset)

The table above reports, for each borough, the **average percentage of English
Language Learners (ELL)** across all records in the `school_demographics` table
that successfully join to the high school directory.

Key points:

- This provides a global view of the ELL population by borough across the full
  time span covered by the data.
- Boroughs with higher `avg_ell_percent` may have a larger concentration of
  English Language Learners and may require more language-related support
  services (e.g., ESL programs, bilingual education).

### 5.3 – Special Education: Top 3 Records per Borough by `sped_percent`

In this section, we want to identify the **three records** in each borough with
the highest percentage of special education students (`sped_percent`).

Our approach:

1. Use `nyc_schools.school_demographics` as the source of `sped_percent`.
2. Join it with `nyc_schools.high_school_directory` on `dbn` to obtain `borough`
   and `school_name`.
3. Use a window function (`ROW_NUMBER()`) to rank records within each borough by
   `sped_percent` in descending order.
4. Select the top 3 ranked records (`rn <= 3`) per borough.

In [12]:
# 5.3 Top 3 records per borough by special education percentage (no aggregation)

q3_sql = """
WITH sped_ranked AS (
    SELECT
        d.borough,
        d.school_name,
        s.dbn,
        s.sped_percent,
        ROW_NUMBER() OVER (
            PARTITION BY d.borough
            ORDER BY s.sped_percent DESC
        ) AS rn
    FROM nyc_schools.school_demographics AS s
    JOIN nyc_schools.high_school_directory AS d
        ON s.dbn = d.dbn
    WHERE s.sped_percent IS NOT NULL
)
SELECT
    borough,
    rn AS borough_rank,
    school_name,
    dbn,
    sped_percent
FROM sped_ranked
WHERE rn <= 3
ORDER BY borough, borough_rank;
"""

q3_df = pd.read_sql(q3_sql, engine)
q3_df

,borough,borough_rank,school_name,dbn,sped_percent
0,Manhattan,1,East Side Community School,01M450,28.8
1,Manhattan,2,East Side Community School,01M450,27.7
2,Manhattan,3,East Side Community School,01M450,26.7


#### Interpretation (Q3)

The table above shows, for each borough, the **three records with the highest
`sped_percent` values**.

Because we do **not** aggregate by school, the same school may appear more than
once if it has multiple records (for example, different school years) among the
top `sped_percent` values in that borough.

This view highlights the most extreme cases of special education concentration
in the dataset, at the record level rather than at the aggregated school level.
If needed, a follow-up analysis could aggregate by school to obtain a single
summary value per school.

## 6. Summary and Insights

This analysis connected to a Neon-hosted PostgreSQL database and used SQL + Python
to explore NYC high school data in the `nyc_schools` schema.

We focused on three questions:

- **Q1 – School Distribution by Borough**  
  Using `high_school_directory` and `COUNT(DISTINCT dbn)`, we computed the number of
  unique schools per borough, ensuring that schools with multiple entries were not
  double-counted.

- **Q2 – Average ELL Percentage by Borough (Full Dataset)**  
  Joining `school_demographics` with `high_school_directory` on `dbn`, we calculated
  the average percentage of English Language Learners (ELL) for each borough,
  using all available records in the demographics dataset (no year filtering).

- **Q3 – Special Education: Top 3 Records per Borough**  
  From the same join, we identified the three records in each borough with the
  highest `sped_percent`. We did not aggregate by school, so the same school may
  appear multiple times if it has multiple high-SPED records.

Overall, these queries provide a concise view of how schools, ELL populations, and
special education concentrations are distributed across boroughs in the available data.